# IMDB RNN, (LSTM, GRU)

RNN은 순서와 맥락을 고려하지는 않음
-> 일치하는 단어가 얼마나 있는가를 확인

## #01. 텍스트 분석 알고리즘

### 1. 텍스트 분석을 위해 개선된 알고리즘의 종류

#### RNN

- Embedding층은 단순하게 데이터의 표현을 학습하여 데이터 사전을 구축하는 것
- 하지만 유사한 의미의 단어를 비슷한 공간에 매핑할 수 있지만, 시퀀스 데이터의 중요한 특성인 순서와 맥락까지 고려한 것은 아니다.
- 순환 신경망은 이 문제를 해결하기 위해 고안된 층
- 완전연결층, 컨볼루션 신경망의 반대되는 개념.
- 완전연결층과 컨볼루션 신경망은 피드 포워드 네트워크(feed-forward network)라고 표현
- 피드 포워드 네트워크는 신경망이 가지는 모든 출력값이 마지막층인 출력층을 향한다.
- 하지만 순환 신경망은 각 층의 결과값이 출력층을 향하면서도 동시에 현재 층의 다음 계산에 사용된다.

#### LSTM

- RNN의 그래디언트 손실문제를 보완한 방법
- 정보를 여러 시점에 걸쳐 나르는 장치(‘Cell state’)가 추가되었다.
- 이로 인해 그래디언트를 보존할 수 있어 그래디언트 손실 문제가 발생하지 않도록 도와준다.

#### GRU

- 게이트 메커니즘이 적용된 RNN의 일종으로 LSTM에서 영감을 받았으며 더 간략한 구조를 갖는다.
- 한국인 조경현 박사님이 제안한 방법

### 2. 텍스트 분석 알고리즘 적용하기

전체 소스코드는 지금까지의 예제들과 동일하게 진행된다.

```
💡 패키지 준비 → 데이터셋 준비 → 데이터 전처리 → 탐색적 데이터 분석(문자열 토큰화, 데이터를 동일한 길이로 맞추기) → 데이터 셋 분할 → 모델 개발(정의+학습) → 학습 결과 평가 → 학습결과 적용
```

이 과정에서 학습 모델을 정의하는 부분에서 적용할 알고리즘만 변경하면 되기 때문에 여기서는 RNN을 먼저 적용해 본 후, 학습 모델을 LSTM과 GRU로 각각 변경하여 다시 학습을 수행해 보도록 한다.

학습 시간이 매우 오래 걸리는 예제이므로 가급적 GPU가 탑재된 컴퓨터에서 실습하는 것이 좋다.

## #02. 패키지 준비하기

In [ ]:
import sys
import os
sys.path.append("/content/drive/MyDrive/DLMLJUPYTER")
import helper_colab
from pandas import DataFrame
import seaborn as sns
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, SimpleRNN # LSTM, GRU
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.datasets import imdb

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## #03. 데이터 준비

In [ ]:
# 예제 데이터
num_words = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = num_words)
print(f"훈련 데이터 {x_train.shape} 레이블 {y_train.shape}")
print(f"훈련 데이터 {x_test.shape} 레이블 {y_test.shape}")

17464789/17464789 [==============================] - 1s 0us/step
훈련 데이터 (25000,) 레이블 (25000,)
훈련 데이터 (25000,) 레이블 (25000,)


## #04. 데이터 전처리

### 1. 데이터를 동일한 길이로 맞추기 (padding)

Embedding층을 사용하기 위해서는 시퀀스 데이터(여기서는 문장을 나타냅니다)의 길이가 전부 동일해야 한다.

길이를 맞추기 위해 pad_sequences() 함수 사용

- 데이터가 지정해준 길이보다 짧은 경우 0으로 채워 넣으며, 긴 경우는 잘라낸다.
- 단어의 뒤에 패딩을 추가하고 싶다면, padding 인자를 post로 지정한다.

> Embedding(임베딩)은 문장의 길이를 동일하게 맞추는 것

padding에는 pre(앞에 0)와 post(뒤에 0)가 존재

- pre-padding일 경우에는 앞단에 0이 채워져 마지막 단어로 제로 패딩이 입력으로 들어가는 일 없이 올바른 시퀀스 모델링이 진행
- post-padding일 경우에는 뒷단에 0이 채워져 마지막 단어로 제로 패딩이 입력으로 사용

In [ ]:
# 최대 문장 길이
max_len = 500

print("padding 이전 :", len(x_train[0]), len(x_train[1]))

pad_x_train = pad_sequences(x_train, maxlen=max_len, padding='pre')
pad_x_test = pad_sequences(x_test, maxlen=max_len, padding='pre')

#  pre-padding으로 원래 단어의 앞에 '지정해준 단어의 길이 – 원래 단어의 길이' (500 –218)만큼 0이 추가된 것을 볼 수 있다.
print('padding 이후 :', len(pad_x_train[0]), len(pad_x_train[1]))

padding 이전 : 218 189
padding 이후 : 500 500


#### pre-padding 결과 확인

In [ ]:
print(pad_x_train[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
print(pad_x_train[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

## #04. 탐색적 데이터 분석

> 실제로는 ***탐색적 데이터분석***을 통해 긍정/부정의 비율이 동일한지 확인 후 모델을 개발하고 학습을 진행. 이 과정을 건너뛰면 모델 생성, 학습이 제대로 되지 않음.

여기서는 동일한 비율로 나눈 예제 데이터를 사용하기 때문에 생략.

## #05. 데이터 셋 분할하기

이미 데이터셋이 훈련데이터와 검증데이터로 분리되어 있기 때문에 별도의 분할 작업을 수행할 필요는 없다.

> 실제로는 **탐색적 데이터분석** 과정에서 긍정/부정의 비율이 동일한지 확인 후 모델을 개발, 학습을 진행. 이 과정을 건너뛰면 모델 생성, 학습이 제대로 되지 않음.

## #06. 모델 개발

### 모델 정의 (RNN)

return_sequences 파라미터가 True로 지정되면 모든 학습 시점의 은닉 상태를 출력해 준다. False인 경우는 마지막 시점의 은닉 상태만 출력한다. (기본값=False)

dropout은 지정된 비율만큼 학습을 건너뛰게 하는 파라미터. 이 파라미터를 사용하게 되면 과거 학습정보를 잃어버릴 확률이 높아지고 그에 따라 모델 성능이 나빠질 가능성이 있다.

>dropout은 무작위로 건너뛰는 비율로 학습 속도는 향상되지만 과거 학습 정보를 잃어버릴 확률이 높아져 모델의 성능 저하가 발생할 수 있음.

recurrent_dropout(순환드롭아웃)은 과거 학습정보를 잃어버리는 문제를 해결하기 위해 적용하는 옵션.
- --------------------

> Embedding층은 모델의 첫 번째 층으로만 사용할 수 있으며, 주로 순환 신경망과 연결하여 사용.

(`Conv2D 적용 [-> BatchNormalization층 적용] -> MaxPool2D 적용 [-> Dropout]`)의 과정을 n번 반복 -> Flatten 레이어 추가 -> Dense층 n개 추가

- **Conv2D 층**
    - 영상이나 이미지의 특징을 확인하기 위한 함수
    - `filters`: 필터(커널)의 개수
    - `kernel_size`: 필터의 크기
    - `strides`: 필터의 이동 간격
    - `padding`: valid(패딩 없음), same(인풋과 아웃풋이 같도록 패딩)
    - `activation`: 활성화 함수
    - `input_shape`: 첫 레이어에 인풋으로 들어오는 크기

- **BatchNormalization (배치정규화)**
    - 신경망 입력데이터를 평균 0, 분산 1로 정규화해서 학습이 잘 이루어지도록 하는 방법
    - BatchNormalization 층에서 정규화를 수행하므로 별도의 정규화를 적용할 필요가 없다.
    - 만약 이 층을 적용하지 않는다면 학습 전에 별도의 정규화를 수행하는 것이 좋다.

- **MaxPool2D**
    - 컨볼루션 레이어의 출력 이미지에서 주요값만 뽑아 크기가 작은 출력 영상을 만든다.
    - `pool_size`: 축소시킬 필터의 크기(비율)
    - `strides`: 필터의 이동 간격. 기본값으로 pool_size를 갖는다.
    - 일반적으로 strides는 pool_size와 동일하게 적용되므로 생략하는 편.

- --------------------
 > filter가 클수록, kernel_size가 작을 수록 모델이 성능이 개선됨.
 단, 학습 시간도 오래 걸림.

> activation : 다중분류 = softmax, unit 10, 이진분류 = sigmoid, unit 1/0 - 주로 `relu`

> loss : 다중분류 = categorical_crossentropy, 이진분류 = binary_crossentropy

> Sequential : 모델을 하나 하나 차례로 집어 넣는다

> Dense : 완전 연결 층

> optimizer : 경사하강법 변화 버전(sgd, adam 등등)

> metrics : 평가지표(학습 판단 기준) mse, mae, acc 등

In [ ]:
model = Sequential()

model.add(Embedding(input_dim = num_words, output_dim = 32, input_length = max_len))
model.add(SimpleRNN(32, return_sequences=True, dropout=0.15, recurrent_dropout=0.15))
model.add(SimpleRNN(16))
model.add(Dense(1, activation='sigmoid'))   #이진분류 = sigmoid

# 결국은 긍정, 부정을 분류 하는 문제이므로 이진분류에 해당한다.
model.compile(optimizer='adam',loss = 'binary_crossentropy', metrics = ['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 32)           320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 500, 32)           2080      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 16)                784       
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 322881 (1.23 MB)
Trainable params: 322881 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### 학습하기

- epochs는 많아도 크게 상관이 없음(단, 너무 크면 오래 걸리고 오차가 증가)
    - callback을 사용해 모델의 학습 방향, 저장 시점, 학습 정지 시점 등에 관한 상황을 모니터링
    - EarlyStopping 콜백과 같이 patience 인자를 지정하여, 지정된 기간 동안 `평가지표에서 성능 향상이 일어나지 않으면 학습률을 조정하는 콜백`
    - ReduceLROnPlateau EarlyStopping 콜백과 같이 patience 인자를 지정하여, 지정된 기간 동안 `평가지표에서 성능 향상이 일어나지 않으면 학습률을 조정하는 콜백`
    - ModelCheckpoint 지정한 평가지표를 기준으로 가장 뛰어난 성능을 보여주는 모델을 저장 할 때 사용

In [ ]:
%%time
result = model.fit(pad_x_train, y_train, epochs=500, validation_data=(pad_x_test, y_test), callbacks = [
    EarlyStopping(monitor = 'val_loss', patience=5, verbose = 1),
    ReduceLROnPlateau(monitor= "val_loss", patience=3, factor = 0.5, min_lr=0.0001, verbose=1)
])

Epoch 1/500
782/782 [==============================] - 1192s 2s/step - loss: 0.3681 - acc: 0.8472 - val_loss: 0.4292 - val_acc: 0.8082 - lr: 0.0010
Epoch 2/500
782/782 [==============================] - 1101s 1s/step - loss: 0.3691 - acc: 0.8478 - val_loss: 0.4690 - val_acc: 0.7671 - lr: 0.0010
Epoch 3/500
103/782 [==>...........................] - ETA: 14:50 - loss: 0.4828 - acc: 0.7715

KeyboardInterrupt: ignored

Epoch 1/500
  3/782 [..............................] - ETA: 13:52 - loss: 0.4301 - acc: 0.8125